In [33]:
import numpy as np
import scipy.misc
import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
import scipy.misc
import os
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

In [34]:
cascPath = "haarcascade_frontalface_default.xml"
# Create the haar cascade
faceCascade = cv2.CascadeClassifier(cascPath)


In [35]:
from skimage.feature import hog#Terrible results; Train and test F1 score of 0 with this. 
from skimage.feature import local_binary_pattern

In [135]:
## read dataset1
full_face = []
full_labels = []
all_images = os.listdir('yalefaces')
for j in range(1,16):    # there are 15 different people in the dataset
    img_list = [filename for filename in all_images if filename.startswith('subject%.2d'%j)]
#     img_list = glob.glob('yalefaces/subject%.2d*'%j)
    img_list=img_list[0:10]#Get only 10 faces of each category
    face1 = np.zeros((len(img_list),64,64))
    for k,img in enumerate(img_list):
        I = scipy.misc.imread(os.path.join('yalefaces',img))
        I = cv2.fastNlMeansDenoising(I, None, 9, 13)
        I = cv2.GaussianBlur(I, (5,5), 0)
        I = cv2.equalizeHist(I)

        faces = faceCascade.detectMultiScale(
            I,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30))
        #fd, hog_image = hog(I, orientations=8, pixels_per_cell=(16, 16),cells_per_block=(1, 1), visualise=True)
        #I = hog_image
        #I = local_binary_pattern(I,8,3)
        try : 
            x,y,h,w = faces[0]
            face1[k,...] = zoom(I[y:y+h,x:x+w],[64./h,64./w])     
        except : 
            x,y,h,w=[0,0,479,639]
            face1[k,...] = zoom(I[y:y+h,x:x+w],[64./h,64./w])
            
        
    labels = (j-1)*np.ones((10,))
    full_face.append(face1)
    full_labels.append(labels)
#full_face = np.stack(full_face)    # contains the images of all faces
#full_labels = np.stack(full_labels)   # contains the ID of all the faces


In [136]:
#full_face = full_face/255 #normalise data

In [137]:
#full_face.shape

In [138]:
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from skimage.feature import hog

In [139]:
#similar 1 dissimilar 0
pairs = []
labels = []
similarity = []
'''
for i in range(0,15):
    for j in range(50):
        a = np.random.randint(0,11,2)
        labels.append([i,i])
        pairs.append(a)
        similarity.append(1)
'''       

'\nfor i in range(0,15):\n    for j in range(50):\n        a = np.random.randint(0,11,2)\n        labels.append([i,i])\n        pairs.append(a)\n        similarity.append(1)\n'

In [140]:
#similar 1 dissimilar 0
'''
for i in range(0,750):
        b = np.random.randint(0,15,2)
        a = np.random.randint(0,11,2)
        labels.append(b)
        pairs.append(a)
        if(b[0]==b[1]):
            similarity.append(1)
        else:
            similarity.append(0)
'''

'\nfor i in range(0,750):\n        b = np.random.randint(0,15,2)\n        a = np.random.randint(0,11,2)\n        labels.append(b)\n        pairs.append(a)\n        if(b[0]==b[1]):\n            similarity.append(1)\n        else:\n            similarity.append(0)\n'

In [141]:
#More data from att_faces
for j in range(16,40):    # there are 40 different people in the dataset
    img_list = []
    for i in range(1,11):
        img_list.append('att_faces/s'+str(j)+'/'+str(i)+'.pgm')
#     img_list = glob.glob('yalefaces/subject%.2d*'%j)
    face1 = np.zeros((len(img_list),64,64))
    for k,img in enumerate(img_list):
        I = scipy.misc.imread(img)
        I = cv2.fastNlMeansDenoising(I, None, 9, 13)
        I = cv2.GaussianBlur(I, (5,5), 0)
        I = cv2.equalizeHist(I)
        faces = faceCascade.detectMultiScale(
            I,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30))
        #fd, hog_image = hog(I, orientations=8, pixels_per_cell=(16, 16),cells_per_block=(1, 1), visualise=True)
        #I = hog_image
        #I = local_binary_pattern(I,8,3)
        try : 
            x,y,h,w = faces[0]
            face1[k,...] = zoom(I[y:y+h,x:x+w],[64./h,64./w])     
        except : 
            x,y,h,w=[0,0,112,92]
            face1[k,...] = zoom(I[y:y+h,x:x+w],[64./h,64./w])   
        
    labels = (j-1)*np.ones((10,))
    full_face.append(face1)
    full_labels.append(labels)
full_face = np.stack(full_face)    # contains the images of all faces
full_labels = np.stack(full_labels)   # contains the ID of all the faces


In [142]:
full_face = full_face/255 #normalise data

In [143]:
len(full_face[10])

10

In [144]:
#similar 1 dissimilar 0
pairs = []
labels = []
similarity = []
for i in range(0,39):
    for j in range(40):
        a = np.random.randint(0,10,2)
        labels.append([i,i])
        pairs.append(a)
        similarity.append(1)
#similar 1 dissimilar 0
for i in range(0,2000):
        b = np.random.randint(0,39,2)
        a = np.random.randint(0,10,2)
        labels.append(b)
        pairs.append(a)
        if(b[0]==b[1]):
            similarity.append(1)
        else:
            similarity.append(0)

In [145]:
X = np.zeros((len(similarity),64,64))

In [146]:
X1 = np.zeros((len(similarity),64*64))
for i in range(len(similarity)):
    a = labels[i]
    b = pairs[i]
    face1 = full_face[a[0],b[0],:,:]
    face2 = full_face[a[1],b[1],:,:]
    face = face1-face2
    face = np.multiply(face,face)
    #face = np.cos(face)
    #face = cosine_similarity(face1,face2)
    X[i,:,:]=face
    X1[i,:]=X[i,:,:].ravel()

In [147]:
xtrain,xtest,ytrain,ytest=train_test_split(X1, similarity, random_state=42, test_size = 0.2)

In [49]:
#Better results without PCA; so don't use this
from sklearn.decomposition import PCA
pca = PCA(whiten=True)
pca.fit(xtrain)
n_components = np.argmin(np.cumsum(pca.explained_variance_ratio_)<=0.99)
print(n_components)
xtrain=pca.transform(xtrain)
xtrain=xtrain[:,:n_components]
xtest=pca.transform(xtest)
xtest=xtest[:,:n_components]

1326


In [85]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(8,10), random_state=1)
clf.fit(xtrain,ytrain)
print('Train Accuracy : ',clf.score(xtrain,ytrain))
print('Test Accuracy : ',clf.score(xtest,ytest))
ypred_train=clf.predict(xtrain)
ypred_test=clf.predict(xtest)
print('Train F1 Score : ',f1_score(y_pred=ypred_train,y_true=ytrain))
print('Test F1 Score : ',f1_score(y_pred=ypred_test,y_true=ytest))

Train Accuracy :  0.8865870786516854
Test Accuracy :  0.827247191011236
Train F1 Score :  0.8729846637829336
Test F1 Score :  0.823021582733813


In [51]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression(C=1000)
clf.fit(xtrain,ytrain)
print('Train Accuracy : ',clf.score(xtrain,ytrain))
print('Test Accuracy : ',clf.score(xtest,ytest))
ypred_train=clf.predict(xtrain)
ypred_test=clf.predict(xtest)
print('Train F1 Score : ',f1_score(y_pred=ypred_train,y_true=ytrain))
print('Test F1 Score : ',f1_score(y_pred=ypred_test,y_true=ytest))

Train Accuracy :  1.0
Test Accuracy :  0.884765625
Train F1 Score :  1.0
Test F1 Score :  0.9115442278860568


In [52]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=400, max_depth=16,random_state=0)#200 estimators gives good results
clf.fit(xtrain,ytrain)
print('Train Accuracy : ',clf.score(xtrain,ytrain))
print('Test Accuracy : ',clf.score(xtest,ytest))
ypred_train=clf.predict(xtrain)
ypred_test=clf.predict(xtest)
print('Train F1 Score : ',f1_score(y_pred=ypred_train,y_true=ytrain))
print('Test F1 Score : ',f1_score(y_pred=ypred_test,y_true=ytest))

Train Accuracy :  1.0
Test Accuracy :  0.85546875
Train F1 Score :  1.0
Test F1 Score :  0.8942857142857144


In [57]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
clf = AdaBoostClassifier(n_estimators=200, base_estimator=RandomForestClassifier(max_depth=6))#200 n_estimators gives best results and took us to the third position on the leaderboard. 
clf.fit(xtrain,ytrain)
print('Train Accuracy : ',clf.score(xtrain,ytrain))
print('Test Accuracy : ',clf.score(xtest,ytest))
ypred_train=clf.predict(xtrain)
ypred_test=clf.predict(xtest)
print('Train F1 Score : ',f1_score(y_pred=ypred_train,y_true=ytrain))
print('Test F1 Score : ',f1_score(y_pred=ypred_test,y_true=ytest))

Train Accuracy :  1.0
Test Accuracy :  0.943359375
Train F1 Score :  1.0
Test F1 Score :  0.9550387596899225


In [86]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
clf = AdaBoostClassifier(n_estimators=200, base_estimator=RandomForestClassifier(max_depth=4))#200 n_estimators gives best results and took us to the third position on the leaderboard. 
clf.fit(xtrain,ytrain)
print('Train Accuracy : ',clf.score(xtrain,ytrain))
print('Test Accuracy : ',clf.score(xtest,ytest))
ypred_train=clf.predict(xtrain)
ypred_test=clf.predict(xtest)
print('Train F1 Score : ',f1_score(y_pred=ypred_train,y_true=ytrain))
print('Test F1 Score : ',f1_score(y_pred=ypred_test,y_true=ytest))

Train Accuracy :  1.0
Test Accuracy :  0.9564606741573034
Train F1 Score :  1.0
Test F1 Score :  0.9546120058565154


In [119]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
clf = AdaBoostClassifier(n_estimators=200, base_estimator=RandomForestClassifier(max_depth=6))#200 n_estimators gives best results and took us to the third position on the leaderboard. 
clf.fit(xtrain,ytrain)
print('Train Accuracy : ',clf.score(xtrain,ytrain))
print('Test Accuracy : ',clf.score(xtest,ytest))
ypred_train=clf.predict(xtrain)
ypred_test=clf.predict(xtest)
print('Train F1 Score : ',f1_score(y_pred=ypred_train,y_true=ytrain))
print('Test F1 Score : ',f1_score(y_pred=ypred_test,y_true=ytest))

Train Accuracy :  1.0
Test Accuracy :  0.9676966292134831
Train F1 Score :  1.0
Test F1 Score :  0.9660265878877401


In [127]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
clf = AdaBoostClassifier(n_estimators=100, base_estimator=RandomForestClassifier(max_depth=6))#200 n_estimators gives best results and took us to the third position on the leaderboard. 
clf.fit(xtrain,ytrain)
print('Train Accuracy : ',clf.score(xtrain,ytrain))
print('Test Accuracy : ',clf.score(xtest,ytest))
ypred_train=clf.predict(xtrain)
ypred_test=clf.predict(xtest)
print('Train F1 Score : ',f1_score(y_pred=ypred_train,y_true=ytrain))
print('Test F1 Score : ',f1_score(y_pred=ypred_test,y_true=ytest))

Train Accuracy :  1.0
Test Accuracy :  0.9648876404494382
Train F1 Score :  1.0
Test F1 Score :  0.962962962962963


In [151]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
clf = AdaBoostClassifier(n_estimators=100, base_estimator=RandomForestClassifier(max_depth=5))#200 n_estimators gives best results and took us to the third position on the leaderboard. 
clf.fit(xtrain,ytrain)
print('Train Accuracy : ',clf.score(xtrain,ytrain))
print('Test Accuracy : ',clf.score(xtest,ytest))
ypred_train=clf.predict(xtrain)
ypred_test=clf.predict(xtest)
print('Train F1 Score : ',f1_score(y_pred=ypred_train,y_true=ytrain))
print('Test F1 Score : ',f1_score(y_pred=ypred_test,y_true=ytest))

Train Accuracy :  1.0
Test Accuracy :  0.9662921348314607
Train F1 Score :  1.0
Test F1 Score :  0.9658119658119657


In [110]:
import h5py
h5f = h5py.File('test.h5','r')
a = list(h5f.keys())
## read the dataset
full_face = []
full_labels = []
img_list=[h5f[i] for i in a]
tface1 = np.zeros((len(img_list),64,64))
for k,img in enumerate(img_list):
    I = np.array(img)#scipy.misc.imread(os.path.join('yalefaces',img))
    #print(type(I))
    I = cv2.fastNlMeansDenoising(I, None, 9, 13)
    I = cv2.GaussianBlur(I, (5,5), 0)
    I = cv2.equalizeHist(I)
    faces = faceCascade.detectMultiScale(
        I,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30))
    #print(k)
    #print(faces)
    #fd, hog_image = hog(I, orientations=8, pixels_per_cell=(16, 16),cells_per_block=(1, 1), visualise=True)
    #I = hog_image
    I = local_binary_pattern(I,8,3)
    try : 
        x,y,h,w = faces[0]
        tface1[k,...] = zoom(I[y:y+h,x:x+w],[64./h,64./w])
    except : 
        x,y,h,w=[0,0,111,91]
        tface1[k,...] = zoom(I[y:y+h,x:x+w],[64./h,64./w])

In [111]:
tface1.shape

(120, 64, 64)

In [112]:
tface1 = tface1/255

In [113]:
test_pairs = np.genfromtxt('image_pairs.csv', dtype=None, delimiter=',', names=True)

In [120]:
euc=[]

for i in range(3540):#(3540):
    x1 = np.zeros((64,64))
    x0 = np.zeros((64,64))
    ids = test_pairs[i][0]
    image1=test_pairs[i][1]
    image2=test_pairs[i][2]
    x0 = tface1[a.index(str(test_pairs[i][1])),:,:]
    x1 = tface1[a.index(str(test_pairs[i][2])),:,:]
    if(i%100==0):
        print(i)
    face = x0-x1
    face = np.multiply(face,face)
    #face = np.cos(face)
    face=face.ravel()
    inp = np.zeros((1,64*64))
    inp[0,:]=face
    #inp=pca.transform(inp)
    #inp=inp[:,:n_components]
    euc.append(clf.predict(inp))

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500


In [121]:
#1 is similar; 0 is different
with open('predictions.csv','w') as f : 
    f.write('Id,Expected\n')
    for i in range(len(test_pairs)) : 
        ids = test_pairs[i][0]
        temp = euc[i][0]
        f.write(str(ids)+','+str(temp)+'\n')

In [122]:
euc = np.asarray(euc)

In [123]:
sum(euc)

array([267])